In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0

    return (x, y), (x_test, y_test)

In [5]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


In [2]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal(shape=(28 * 28, 128), stddev=0.01), trainable=True)
        self.b1 = tf.Variable(tf.zeros(shape=(128,)), trainable=True)
        self.W2 = tf.Variable(tf.random.normal(shape=(128, 10), stddev=0.01), trainable=True)
        self.b2 = tf.Variable(tf.zeros(shape=(10,)), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x_flattened = tf.reshape(x, [-1, 784])
        h1 = tf.matmul(x_flattened, self.W1) + self.b1
        activated_h1 = tf.tanh(h1)
        logits = tf.matmul(activated_h1, self.W2) + self.b2
        return logits

model = myModel()

optimizer = optimizers.Adam()

In [3]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

In [8]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer,
                                    tf.constant(train_data[0], dtype=tf.float32),
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model,
                      tf.constant(test_data[0], dtype=tf.float32),
                      tf.constant(test_data[1], dtype=tf.int64))



print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.2183914 ; accuracy 0.61535
epoch 1 : loss 2.2176483 ; accuracy 0.6153
epoch 2 : loss 2.2168994 ; accuracy 0.61555
epoch 3 : loss 2.2161453 ; accuracy 0.61581665
epoch 4 : loss 2.215386 ; accuracy 0.61585
epoch 5 : loss 2.2146208 ; accuracy 0.61618334
epoch 6 : loss 2.21385 ; accuracy 0.6165
epoch 7 : loss 2.213074 ; accuracy 0.6167333
epoch 8 : loss 2.2122922 ; accuracy 0.61681664
epoch 9 : loss 2.211505 ; accuracy 0.61721665
epoch 10 : loss 2.210712 ; accuracy 0.61755
epoch 11 : loss 2.209913 ; accuracy 0.61775
epoch 12 : loss 2.2091086 ; accuracy 0.61791664
epoch 13 : loss 2.2082984 ; accuracy 0.61808336
epoch 14 : loss 2.2074823 ; accuracy 0.61841667
epoch 15 : loss 2.2066605 ; accuracy 0.61838335
epoch 16 : loss 2.2058322 ; accuracy 0.61841667
epoch 17 : loss 2.2049985 ; accuracy 0.6185
epoch 18 : loss 2.2041588 ; accuracy 0.61855
epoch 19 : loss 2.203313 ; accuracy 0.61863333
epoch 20 : loss 2.2024615 ; accuracy 0.61865
epoch 21 : loss 2.2016034 ; accuracy 0.61856